In [ ]:
!git clone https://github.com/huggingface/diffusers

In [ ]:
%cd diffusers

In [ ]:
!pip install -e .

In [ ]:
!pip install -r examples/dreambooth/requirements_sdxl.txt

In [ ]:
!pip install bitsandbytes

In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()

In [ ]:
import wandb
wandb.login(key = "wandb_api_key") # replace with your wandb key

In [ ]:
!accelerate launch examples/dreambooth/train_dreambooth_lora_sdxl.py \
    --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
    --instance_data_dir="image_data_dir" \
    --output_dir="/kaggle/working/irit_lora" \
    --mixed_precision="bf16" \
    --instance_prompt="a photo of owhn woman" \
    --seed="0" \
    --resolution=1024 \
    --train_batch_size=1 \
    --gradient_accumulation_steps=1 \
    --learning_rate=1e-4 \
    --report_to="wandb" \
    --lr_scheduler="constant" \
    --lr_warmup_steps=0 \
    --num_train_epochs=20 \
    --validation_epochs=4 \
    --num_validation_images=2 \
    --repeats=5 \
    --checkpointing_steps=180 \
    --seed="0" \
    --use_8bit_adam \
    --rank=64 \
    --gradient_checkpointing \
    --train_text_encoder \
    --text_encoder_lr=5e-6 \
    --validation_prompt="a professional photo portrait of owhn woman holding flowers, model photoshoot, professional photo, white background, Amazing Details, Best Quality, Masterpiece, dramatic lighting highly detailed, analog photo, overglaze"

In [ ]:
%cd /kaggle/working

In [13]:
!rm -rf diffusers

In [14]:
import sys

# Xoá module diffusers khỏi cache
if "diffusers" in sys.modules:
    del sys.modules["diffusers"]

# Loại bỏ tất cả đường dẫn trỏ đến local diffusers
sys.path = [p for p in sys.path if "diffusers" not in p]

In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git --quiet

In [ ]:
from diffusers import DiffusionPipeline
import torch
base_model_id = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = DiffusionPipeline.from_pretrained(base_model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")
lora_model_id = "/kaggle/working/irit_lora"
pipe.load_lora_weights(lora_model_id)
prompt = "a professional photo portrait of owhn woman holding flowers, model photoshoot, professional photo, white background, Amazing Details, Best Quality, Masterpiece, dramatic lighting highly detailed, analog photo, overglaze, 80mm Sigma f/1.4 or any ZEISS lens"
negative_prompt = "asian, , , multiple heads, 2 heads, elongated body, double image, 2 faces, multiple people, double head, , (nsfw)+, naked, nude, porn, (deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up"
image = pipe(prompt, negative_prompt = negative_prompt, num_inference_steps=50).images[0]

In [ ]:
image

In [ ]:
import os 
import numpy as np 
output_dir = "/kaggle/working/generated_images"
os.makedirs(output_dir, exist_ok=True)
prompt = "a professional photo portrait of owhn woman holding flowers, model photoshoot, professional photo, white background, Amazing Details, Best Quality, Masterpiece, dramatic lighting highly detailed, analog photo, overglaze, 80mm Sigma f/1.4 or any ZEISS lens"
negative_prompt = "asian, , , multiple heads, 2 heads, elongated body, double image, 2 faces, multiple people, double head, , (nsfw)+, naked, nude, porn, (deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up"
for i in range(20):
    seed = np.random.randint(0, 999999)
    generator = torch.manual_seed(seed)
    image = pipe(prompt, negative_prompt = negative_prompt, num_inference_steps=100, guidance_scale=8, generator=generator).images[0]
    image.save(f"{output_dir}/image_{i+1:02d}_seed{seed}.png")